In [210]:
import pandas as pd
import numpy as np

# VSB : close rate (ponderate by outbound win opportunities) and total arpa_vsb (ponderate by total opportunities) :

step 1 : getting the data (csv extracted from https://payfit.eu.looker.com/dashboards/1233?CCN%20Code%20and%20Name=-CCN%20non%20r%C3%A9f%C3%A9renc%C3%A9e%20ou%20pas%20de%20CCN,-N%2FA&PayFit%20Clients%20per%20CCN=%3E%3D1&D%C3%A9partement=-97%25&INSEE%20Employee%20Tier=VSB (looker dashboard filtered on VSB))

In [211]:
df_closerate_vsb = pd.read_csv('closerate_vsb.csv', header = 1)
df_arpa_vsb = pd.read_csv('arpa_vsb.csv', header=1)

## Close rate and arpa_vsb for VSB

###  drop na and rename / drop columns

In [212]:
df_closerate_vsb = df_closerate_vsb.drop(columns = ["Close Ratio",'Unnamed: 0'])
df_closerate_vsb = df_closerate_vsb.rename(columns={"Close Ratio.1": "close_ratio_ob"})
df_closerate_vsb = df_closerate_vsb.dropna()
df_arpa_vsb = df_arpa_vsb.dropna()
df_arpa_vsb = df_arpa_vsb.drop(columns = ['Unnamed: 0','Won Opportunities','ARPA ','ARPA .1'])
df_arpa_vsb = df_arpa_vsb.rename(columns={'Won Opportunities.1': "won_opportunities_ob", 'Won Opportunities.2': 'won_opportunities_tot', 'ARPA .2':'arpa_tot'})

### collective_agreement to Syntec/HCR/RR

In [213]:
def cag_to_ccn (x) :
    if '1486' in x.lower() or 'syntec' in x.lower() :
        return ('SYNTEC')
    elif '1979' in x or 'restaurant' in x.lower() :
        return ('HCR')
    elif '1501' in x or 'restauration' in x.lower() :
        return('RR')
    else :
        return(x)


In [214]:
df_closerate_vsb['Collective Agreement_process'] = df_closerate_vsb['Collective Agreement'].apply(cag_to_ccn)
df_arpa_vsb['Collective Agreement_process'] = df_arpa_vsb['Collective Agreement'].apply(cag_to_ccn)

### obtention of filtered table containing only SYNTEC, HCR and RR datas

In [215]:
df_closerate_filtered_vsb = df_closerate_vsb[df_closerate_vsb['Collective Agreement_process'].isin(['SYNTEC','HCR','RR'])]
df_arpa_filtered_vsb = df_arpa_vsb[df_arpa_vsb['Collective Agreement_process'].isin(['SYNTEC','HCR','RR'])]

merging of the two tables on Collective agreement

In [216]:
df_merge_filtered_vsb = df_arpa_filtered_vsb.merge(df_closerate_filtered_vsb, how= 'left', on = 'Collective Agreement')
df_merge_filtered_vsb.drop(columns='Collective Agreement_process_y',inplace=True)

### creation of arpa_vsb_cumul and close_rate_ob_cumul

conversion of columns to int / float :

In [217]:
def euros_to_int(x):
    return(int(x.split('€')[0]))

df_merge_filtered_vsb['arpa_tot'] = df_merge_filtered_vsb['arpa_tot'].apply(euros_to_int)
df_merge_filtered_vsb['close_ratio_ob'] = df_merge_filtered_vsb['close_ratio_ob'].apply(lambda x: float(str(x).split('%')[0]))
df_merge_filtered_vsb['won_opportunities_tot'] = df_merge_filtered_vsb['won_opportunities_tot'].apply(lambda x: int(x))

creation of arpa_vsb_cumul and close_rate_ob_cumul

In [218]:
df_merge_filtered_vsb['arpa_cumul'] = df_merge_filtered_vsb['won_opportunities_tot']* df_merge_filtered_vsb['arpa_tot']
df_merge_filtered_vsb['close_ratio_ob_cumul'] = df_merge_filtered_vsb['won_opportunities_ob'] * df_merge_filtered_vsb['close_ratio_ob']

### group by collective_agreement process and sum

In [219]:
df_merge_filtered_vsb_group = df_merge_filtered_vsb.groupby('Collective Agreement_process_x').sum()

### Obtention of final insights

In [220]:
df_merge_filtered_vsb_group['arpa_vsb_final'] = df_merge_filtered_vsb_group['arpa_cumul'] / df_merge_filtered_vsb_group['won_opportunities_tot']
df_merge_filtered_vsb_group['close_rate_vsb_final'] = df_merge_filtered_vsb_group['close_ratio_ob_cumul'] / df_merge_filtered_vsb_group['won_opportunities_ob']

# SB (same process)

In [221]:
df_closerate_sb = pd.read_csv('closerate_sb.csv', header = 1)
df_arpa_sb = pd.read_csv('arpa_sb.csv', header=1)

In [222]:
df_closerate_sb = df_closerate_sb.drop(columns = ["Close Ratio",'Unnamed: 0'])
df_closerate_sb = df_closerate_sb.rename(columns={"Close Ratio.1": "close_ratio_ob"})
df_closerate_sb = df_closerate_sb.dropna()
df_arpa_sb = df_arpa_sb.dropna()
df_arpa_sb = df_arpa_sb.drop(columns = ['Unnamed: 0','Won Opportunities','ARPA ','ARPA .1'])
df_arpa_sb = df_arpa_sb.rename(columns={'Won Opportunities.1': "won_opportunities_ob", 'Won Opportunities.2': 'won_opportunities_tot', 'ARPA .2':'arpa_tot'})

In [223]:
df_closerate_sb['Collective Agreement_process'] = df_closerate_sb['Collective Agreement'].apply(cag_to_ccn)
df_arpa_sb['Collective Agreement_process'] = df_arpa_sb['Collective Agreement'].apply(cag_to_ccn)

In [224]:
df_closerate_filtered_sb = df_closerate_sb[df_closerate_sb['Collective Agreement_process'].isin(['SYNTEC','HCR','RR'])]
df_arpa_filtered_sb = df_arpa_sb[df_arpa_sb['Collective Agreement_process'].isin(['SYNTEC','HCR','RR'])]

In [225]:
df_merge_filtered_sb = df_arpa_filtered_sb.merge(df_closerate_filtered_sb, how= 'left', on = 'Collective Agreement')
df_merge_filtered_sb.drop(columns='Collective Agreement_process_y',inplace=True)

In [226]:
df_merge_filtered_sb['arpa_tot'] = df_merge_filtered_sb['arpa_tot'].apply(euros_to_int)
df_merge_filtered_sb['close_ratio_ob'] = df_merge_filtered_sb['close_ratio_ob'].apply(lambda x: float(str(x).split('%')[0]))
df_merge_filtered_sb['won_opportunities_tot'] = df_merge_filtered_sb['won_opportunities_tot'].apply(lambda x: int(x))

In [227]:
df_merge_filtered_sb['arpa_cumul'] = df_merge_filtered_sb['won_opportunities_tot']* df_merge_filtered_sb['arpa_tot']
df_merge_filtered_sb['close_ratio_ob_cumul'] = df_merge_filtered_sb['won_opportunities_ob'] * df_merge_filtered_sb['close_ratio_ob']

In [228]:
df_merge_filtered_sb_group = df_merge_filtered_sb.groupby('Collective Agreement_process_x').sum()

In [229]:
df_merge_filtered_sb_group['arpa_vsb_final'] = df_merge_filtered_sb_group['arpa_cumul'] / df_merge_filtered_sb_group['won_opportunities_tot']
df_merge_filtered_sb_group['close_rate_vsb_final'] = df_merge_filtered_sb_group['close_ratio_ob_cumul'] / df_merge_filtered_sb_group['won_opportunities_ob']

# MM